# Solo work for week 3

## Word meaning in the seasons corpus

In class, we built a termxdocument matrix of the seasons data. Each *column* of that matrix corresponded to a document - it was the document vector for that document. In essence, we were understanding each column as the *meaning* of the corresponding document.

Now get ready to have your mind blown: Just as we understood each column in the termxdocument matrix as the meaning of a document, we can understand each row in the documnet as a meaning of the corresponding *term*. The idea here is that the meaning of a term can be understood as a union of all of the context in which that word appears.

You are going to explore this idea a bit. I actually haven't tried this myself, so I'm not sure anything interesting will fall out. Maybe that's a poor way to design an instructional activity. I think it's more fun this way.

### To make your lives easier, I have copied over the first parts of notebook 10.

**Load the corpus**

In [ ]:
from seasons_module import load_seasons_corpus
seasons_corpus = load_seasons_corpus()

**Compile the vocabulary in the usual way.**

In [ ]:
set_vocab = set([])
for fname in seasons_corpus.keys():
    set_vocab = set_vocab.union(set(seasons_corpus[fname][0]))
f = open("lists/seasons_stop_list.txt")
stop_list = set(f.read().split("\n"))
pruned_vocab = set(sorted([w for w in list(set_vocab) if w not in stop_list]))

**Compute the corpus and document frequency for each term.**

In [ ]:
word_fdist = nltk.FreqDist() # the corpus frequences
doc_fdist = nltk.FreqDist()# the document frequencies
for word in pruned_vocab:
    word_fdist[word] = 0
    doc_fdist[word] = 0
    for name in seasons_corpus.keys():
        if word in seasons_corpus[name][0]:
            doc_fdist[word] += 1
            word_fdist[word] += seasons_corpus[name][0].count(word)

**Create a very small vocabulary**

Just 10 words, to make it more simple to understand what's going on.

In [ ]:
small_vocab = [w[0] for w in word_fdist.most_common(10)]
print(small_vocab)

**Compute the weighted document vector for each document**

Same as before, but now using our smaller vocabulary.

In [ ]:
def tf(tf, df, cf, N):
    return tf

def logtf(tf, df, cf, N):
    if tf == 0:
        result = 0
    else:
        result = (1 + np.log(tf))
    return result

def onehot(tf, df, cf, N):
    if tf == 0:
        return 0
    else:
        return 1

def tfidf(tf, df, cf, N):
    if tf == 0:
        result = 0
    else:
        result = (1 + np.log(tf)) * np.log(N  / df)
    return result

def compute_vector(words, vocab, df, N, weight_function):
    new_vector = []
    for w in vocab:
        tf = words.count(w)
        new_vector.append(weight_function(tf, df[w], 0, N))
    return norm_vec(np.array(new_vector))

In [ ]:
# compute the document vector for each document
doc_vectors = {}
N = len(seasons_corpus.keys())
wf = tf
for fname in seasons_corpus.keys():
    doc_vectors[fname] = compute_vector(seasons_corpus[fname][0], small_vocab, doc_fdist, N, wf)

In [ ]:
print(len(doc_vectors), len(doc_vectors['angelapre']))

**Create a termxdocument matrix**

This is a matrix where every row corresponds to a word in the vocabulary, and every column corresponds to a document.

Another way to say this: Each column in the matrix is the document vector for a document.

In [ ]:
td_matrix = np.zeros([len(small_vocab), len(doc_vectors)])
i = 0
name_index = {}
name_list = []
for name in doc_vectors.keys():
    td_matrix[:, i] = doc_vectors[name]
    name_index[name] = i
    name_list += [name]
    i = i + 1

### Here is where things get new

The line below is what we executed to compute a matrix reflecting the documentxdocument similarity. You want to modify this line to that it instead computes a matrix showing the termxterm similarity. Let's call it `tt_matrix`

In [ ]:
dd_matrix = np.dot(td_matrix.transpose(), td_matrix)

Once you have the termxterm similarity, here are some additional things to do. Go as far as you like in these tasks. If you get board of this, you can try another set of tasks I'll put below.

1. Look at `tt_matrix`. Are the term similarities what you expected?
2. It might help to make a heatmap of the sort we made when looking at word co-occurrences. Try that.
3. Above we used only the 10 most common terms. Expand this a bit to include some more terms. You might want to think a bit about what terms would be interesting to compare, and make sure that you have included those terms.

And if you're really ambitious:

4. In this exercise, we are understanding a term's meaning as the collection of contexts in which it appears. So far we have been defining a context simply as the entire document in which the word appears. But there are many other ways we could define a context. The task here is to try some other contexts. One possibility is to look at the collection of utterances in which the word appears. In that case, we'd be treating each utterance as a document when we make are termxdocument matrix. To facilitate this a bit, I made a function that loads the corpus for you, but gives each transcript as a list of tokenized utterances. It's invoked in the next cell.

In [ ]:
from seasons_module import load_seasons_corpus_as_utterances
seasons_corpus_with_utterances = load_seasons_corpus_as_utterances()

## Paragraph-paragraph similarity as a measure of text difficulty

Here's something completely different you can try, if you're board of the seasons corpus.

Our task here is to develop a measure of how difficult a text is by looking at the inter-paragraph cohesion. 
The idea is that we'll:

* construct a vector for every sentence or paragraph in a text. 
* find the similarity between sequential pair of paragraphs
* and, finally, we'll find the averages of these similarities.

If the average of the similarities is high (i.e., closer to one) then the cohesion is high.

I'm think that you'd pick a couple of text from the Gutenberg corpus, and compare their cohesions.

The first step will be to read in one of these corpora. For example

In [16]:
import nltk
mycorpus = nltk.corpus.PlaintextCorpusReader("corpora/gutenberg", 'melville-moby_dick.txt')

Once you've got this, you can access the book as paragraphs or sentences. For example, this gives you the paragraphs:

In [25]:
mycorpus.paras()

[[['[', 'Moby', 'Dick', 'by', 'Herman', 'Melville', '1851', ']']], [['ETYMOLOGY', '.']], ...]

Notice that `mycorpus.paras()` give you a list, where each item in the list is a paragraph. 
But each paragraph is itself a list of sentences.

Probably a first step then is to combine the sentences in each paragraph into one long list of words.
I guess I'll do that for you:

In [28]:
paragraphs = []
for para in mycorpus.paras():
    flat_para = []
    for sent in para:
        flat_para += sent
    paragraphs.append(flat_para)

Now `paragraphs` is a long list of paragraphs.
The next step is to convert each of these paragraphs into a vector:

Next find the dot product between each successive pair of paragraphs.
Add them up.
Then divide by the total number of paragraphs.

That's your measure of coherence